# Test Gradient CIPL

In [1]:
%matplotlib inline
import os
import sys

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import scipy
from sklearn import svm, cross_validation
from theano import config, shared
import theano.tensor as T

sys.path.insert(0, os.path.abspath('../..'))

plt.style.use('fivethirtyeight')  # Style plots

from deepensemble.utils import *

## Generate Data:

In [2]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=6, n_informative=4,
                           n_classes=2, n_clusters_per_class=3, random_state=0)

classes_labels = [d for d in range(2)]
X = np.array(X, dtype=config.floatX)
y = np.array(y[:,np.newaxis], dtype=config.floatX)

# Silverman
s = ITLFunctions.silverman(shared(np.array(y)), len(y), len(classes_labels)).eval()

## Gradient CIP

In [3]:
wb = np.sqrt(6.0 / (6 + 1))

W1 = np.array(np.random.uniform(low=-wb, high=wb, size=(X.shape[1], y.shape[1])), dtype=config.floatX)
W2 = np.array(np.random.uniform(low=-wb, high=wb, size=(X.shape[1], y.shape[1])), dtype=config.floatX)
W3 = np.array(np.random.uniform(low=-wb, high=wb, size=(X.shape[1], y.shape[1])), dtype=config.floatX)

b1 = np.array(np.zeros(size=(y.shape[1], y.shape[1])), dtype=config.floatX)
b2 = np.array(np.zeros(size=(y.shape[1], y.shape[1])), dtype=config.floatX)
b3 = np.array(np.zeros(size=(y.shape[1], y.shape[1])), dtype=config.floatX)

lr = 0.1
max_epoch = 100
    
for i in range(max_epoch):
    # Forward:
    y1 = W1 * X + b1
    y2 = W2 * X + b2
    y3 = W3 * X + b3
    
    # Cost:
    YY = [y1, y2, y3, y]
    
    V_nc, V_J, V_M = ITLFunctions.get_cip_numpy(YY, s)
    
    V_ED = V_J - 2 * V_nc + V_M
    
    # Gradient:
    dV_nc, dV_J, dV_M = ITLFunctions.get_grad_cip_numpy(YY, s)

TypeError: Required argument 'shape' (pos 1) not found